In [9]:
import sys
import glob
import os
import cv2
import numpy as np
from __future__ import absolute_import

import tensorflow as tf
print ("tensorflow v:" + tf.__version__)


from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
#!conda install --yes --prefix {sys.prefix} opencv
#!{sys.executable} -m pip install opencv-contrib-python
#conda install -c conda-forge opencv

tensorflow v:1.12.0


In [6]:
import sys
!conda install --yes --prefix {sys.prefix} keras


Solving environment: done

# All requested packages already installed.



In [10]:

#images = []

skinny_j = []
straight_j = []
bootc = []
def find(obj, l):
    dir1 = "/Users/Simron/ds/examples/pants/proj/images/" + obj
   
    
    #for x in os.walk(dir1):
            #print x
    m_dir= glob.glob(dir1 + "/*/*jpg")
    for files in m_dir:
            image = cv2.imread(files, 1)
            print image.shape
            
            #print image.shape
            #asp = 100.0 / image.shape[1]
            #dim = (100, int(image.shape[0]*asp))
            
            #resize = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            #print resize.shape
            #l.append(resize)
            l.append(image)
    l = np.array(l)
    #print ("shape:", l.shape)
    return l
    
bootc = find("bc", bootc)
skinny_j = find("skinny", skinny_j)
straight_j = find("straight", straight_j)

y_bc = np.full((bootc.shape[0], 1), 0)
y_skj = np.full((skinny_j.shape[0], 1), 1)
y_sj = np.full((straight_j.shape[0], 1), 2)



(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 392, 3)
(459, 

(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(453, 302, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(333, 222, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 

(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 247, 3)
(329, 

All of the images come of different shapes and sizes so rather than downsampling certain images and risking the loss of data, we will resize and pad certain images and compare which is better.

In [45]:
#730 x 480

def resize(l, size):
    ret_l = []   
    count = 0
    for i in range(len(l)):        
        o_size = l[i].shape[:2]

        ratio = float(size)/max(o_size)
        n_size = tuple([int(x*ratio) for x in o_size])

        im = l[i]
        im = cv2.resize(im, (n_size[1], n_size[0]))

        delta_w = size - n_size[1]
        delta_h = size - n_size[0]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)
        color = [0, 0, 0]
        new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        #print new_im.shape
        ret_l.append(new_im)
    #for e in np.array(ret_l):
            #print e.shape
    return np.array(ret_l)

In [12]:
bootc_r = resize(bootc, 730)


In [16]:
skinny_r = resize(skinny_j, 730)
straight_r = resize(straight_j, 730)

In [17]:
model = Sequential()
#Convolution
model.add(Conv2D(32, (3, 3), input_shape = (730, 730, 3), activation = 'relu'))

#Pooling
model.add(MaxPooling2D(pool_size = (2, 2)))


# Adding a second convolutional layer
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))


# Flattening
model.add(Flatten())

# Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [40]:


X = np.vstack([bootc_r, skinny_r, straight_r])
y = np.vstack([y_bc, y_skj, y_sj])

#print ("x shape:" + X.shape)
#print ("y shape:" + y.shape)


TypeError: cannot concatenate 'str' and 'tuple' objects

In [37]:
#data = np.hstack([X, y.T])
#data = X + y.T
#i = 0
#data = np.array([])
#for e in X:
    #tup = (e, y[i])
    #i = i+1
    #data = np.append(data, tup)
#X2 = np.array([])
#np.random.shuffle(data)
#X = data[:, 0: data.shape[1]-1]
#y = data[:, data.shape[1]-1:data.shape[1]]


(1,)
(730, 730, 3)


ValueError: setting an array element with a sequence.

In [41]:
print (X.shape)
print (y.shape)


(1675, 730, 730, 3)
(1675, 1)


Now we shall normalize the data by subtracting the mean of the samples from each sample. Additionally we will divide by the standard deviation of each pixel. Subtracting the mean centers the input to 0, and dividing by the standard deviation makes any scaled feature value the number of standard deviations away from the mean.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
print "hello1"
scaler.fit(X)
print ("hello2")
X_sc = scaler.transform(X)

#X = X - np.mean(X, axis=0)

#X = X/np.std(X, axis=0)

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, X, y, cv=5)

cv_scores.mean()

## LOOCV

y_pred = []

for i in range(len(X)):
    abi = range(0, i) + range(i+1, len(X))
    X_train = X[abi]
    y_train = y[abi]
    
    model.fit(X_train, y_train, shuffle=True, epochs=150)
    X_test = X[i]
    y_test = y[i]
    y_act = model.predict(X_test)
    y_pred.append(y_act)
accuracy = 0
for i in range(len(X)):
    if (y_pred[i] == y[i]):
        accuracy = accuracy + 1
        
print ("accuracy : %d / %d" % (accuracy, len(X)))